<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Saving-and-Loading-Models" data-toc-modified-id="Saving-and-Loading-Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Saving and Loading Models</a></span><ul class="toc-item"><li><span><a href="#Making-a-Basic-Model" data-toc-modified-id="Making-a-Basic-Model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Making a Basic Model</a></span></li><li><span><a href="#Whole-Model-Saving-Model.save()" data-toc-modified-id="Whole-Model-Saving-Model.save()-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Whole Model Saving <code>Model.save()</code></a></span><ul class="toc-item"><li><span><a href="#H5-Format" data-toc-modified-id="H5-Format-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>H5 Format</a></span></li><li><span><a href="#SavedModel-Format" data-toc-modified-id="SavedModel-Format-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>SavedModel Format</a></span></li><li><span><a href="#Loading-An-Entire-Model-keras.models.load_model()" data-toc-modified-id="Loading-An-Entire-Model-keras.models.load_model()-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Loading An Entire Model <code>keras.models.load_model()</code></a></span></li></ul></li><li><span><a href="#Saving-Model-Architecture-and-Weights" data-toc-modified-id="Saving-Model-Architecture-and-Weights-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Saving Model Architecture and Weights</a></span></li><li><span><a href="#Saving-Subclassed-Models-(Clunky)" data-toc-modified-id="Saving-Subclassed-Models-(Clunky)-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Saving Subclassed Models (Clunky)</a></span></li><li><span><a href="#Model-Checkpointing" data-toc-modified-id="Model-Checkpointing-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Model Checkpointing</a></span><ul class="toc-item"><li><span><a href="#Checkpointing-Customization" data-toc-modified-id="Checkpointing-Customization-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Checkpointing Customization</a></span></li></ul></li><li><span><a href="#Using-SavedModel-Objects" data-toc-modified-id="Using-SavedModel-Objects-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Using SavedModel Objects</a></span></li><li><span><a href="#Concrete-Functions" data-toc-modified-id="Concrete-Functions-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Concrete Functions</a></span></li><li><span><a href="#Converting-TF2-Models-to-PyTorch" data-toc-modified-id="Converting-TF2-Models-to-PyTorch-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Converting TF2 Models to PyTorch</a></span></li></ul></li></ul></div>

# Saving and Loading Models

Remember the golden rules, 
- __learn concepts then tools__
- __practice everyday__


In [ ]:
#!pip install --upgrade tensorflow

In [1]:
import os
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
tf.config.list_physical_devices('GPU') #checks for GPU

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Making a Basic Model


In [3]:
#Lad the MNIST toy dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#Vectorize the data
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255


# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

# Create a basic model instance
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
hist = model.fit(x=x_train, y=y_train, batch_size=128, epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 2s 36us/sample - loss: 0.2888 - accuracy: 0.9171


## Whole Model Saving `Model.save()`
[Save and serialize models with Keras
](https://www.tensorflow.org/guide/keras/save_and_serialize)

Save the model in its entirety. Including:
- model architecture
- model weights
- model optimizer, loss function, metrics (training configuration)

Saved in either __Keras H5 format__ or __TensorFlow SavedModel Format__


### H5 Format
Provide a filepath that ends in `.h5`. The model will be saved to a single binarized file. (Works for Keras Sequential Models or Keras Functional API Models )

In [5]:
h5_save_path = 'E://Models/MNIST_H5-Format.h5'
model.save(h5_save_path, save_format='h5') #h5 format

### SavedModel Format
[Docs](https://www.tensorflow.org/guide/saved_model)
Need to create a directory for the model to be saved to. This save serialization is unique to TensorFlow.

In [6]:
sm_save_path = 'E://Models/MNIST_SavedModel-Format/'
model.save(sm_save_path, save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: E://Models/MNIST_SavedModel-Format/assets


### Loading An Entire Model `keras.models.load_model()`

*Saved models can be reinstantiated via `keras.models.load_model`.
The model returned by `load_model` is a compiled model ready to be used
(unless the saved model was never compiled in the first place).*


In [7]:
del model
model = tf.keras.models.load_model(h5_save_path) #H5 file
# Evaluate the model
loss, acc = model.evaluate(x_test,  y_test, verbose=0)
print("Load Model Accuracy: {:5.2f}%".format(100*acc))

Load Model Accuracy: 96.11%


In [8]:
del model
model = tf.keras.models.load_model(sm_save_path) #directory
# Evaluate the model
loss, acc = model.evaluate(x_test,  y_test, verbose=0)
print("Load Model Accuracy: {:5.2f}%".format(100*acc))

Load Model Accuracy: 96.11%


## Saving Model Architecture and Weights
[Architecture-only saving](https://www.tensorflow.org/guide/keras/save_and_serialize#architecture-only_saving)

[Weights-only saving](https://www.tensorflow.org/guide/keras/save_and_serialize#weights-only_saving)

In [ ]:
#save model architecture to JSON
json_config = model.to_json()
with open("E://Models/MNIST_config.json", 'w') as f:
    f.write(json_config)

#save model weights to H5
fpath = "E://Models/MNIST_Weights.h5"
weights = model.save_weights(fpath, save_format='h5')    
  

In [9]:
#load this to load architecture from json
with open("E://Models/MNIST_config.json") as f:
    json_config = f.read()
    
model = tf.keras.models.model_from_json(json_config)

#Then load weights to arch from H5 file.
model.load_weights(fpath)

## Saving Subclassed Models (Clunky)
[Saving Subclassed Models](https://www.tensorflow.org/guide/keras/save_and_serialize#saving_subclassed_models)

*Until the model has been called, it does not know the shape and dtype of the input data it should be expecting, and thus cannot create its weight variables. You may remember that in the Functional model from the first section, the shape and dtype of the inputs was specified in advance (via keras.Input(...)) -- that's why Functional models have a state as soon as they're instantiated.*

__Subclassed model parameters are not initialized until .fit()__

In [ ]:
class subclassedModel(tf.keras.Model):
    def __init__(self, name=None):
        super(subclassedModel, self).__init__(name=name)
        self.dense1 = tf.keras.layers.Dense(64, activation='relu', name='dense1')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu', name='dense2')
        self.dense3 = tf.keras.layers.Dense(10, activation='softmax', name='dense3')
        
    def call(self, x):  #call method defined with the functional API
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)      

In [ ]:
model =  subclassedModel(name='TheGreatModel')
model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics = ['acc'])
hist = model.fit(x_train, y_train, batch_size=128, epochs=1)

Need to save the weights, since the model is defined by the code. `model.save_weights` will save:
- layer weights
- optimizer state
- stateful model metric variables

This is SavedModel format

In [ ]:
save_path = 'E://Models/MNIST_SubClassedModel_2-10-20/'
model.save_weights(save_path, save_format='tf')

In [ ]:
del model
model = subclassedModel(name='TheGreatModel')
model.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics = ['acc'])
# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
model.train_on_batch(x_train[:1], y_train[:1])

# Load the state of the old model
model.load_weights(save_path)

## Model Checkpointing

[Training checkpoints](https://www.tensorflow.org/guide/checkpoint)

The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both during and at the end of training.

__Note that saving an entire model will fail unless you end the save_path as__ `save/path/` not `save/path`

In [ ]:
checkpoint_path = 'E://Models/MNIST_Checkpoints_2-10-20/cp1/'

# Create a callback that saves the model's weights (SavedModel Format)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 #save_weights_only=True,
                                                 verbose=1)

model = create_model()

# Train the model with the new callback
hist = model.fit(x_train, 
          y_train,  
          epochs=10,
          verbose=0,
          validation_data=(x_test,y_test),
          callbacks=[cp_callback])  # Pass callback to training

In [ ]:
# Loads the weights
model = tf.keras.models.load_model(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(x_test,  y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Checkpointing Customization
Save weights only every 5 Epochs. This is the __TensorFlow Checkpoint format__

In [ ]:
checkpoint_path = 'E://Models/MNIST_Checkpoints_2-10-20/cp2/model_weights_epoch_{epoch}.ckpt'

cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                       save_weights_only=True,
                                       period=5,
                                       verbose=1)

model = create_model()
# Train the model with the new callback
hist = model.fit(x_train, 
          y_train,  
          epochs=10,
          verbose=0,
          validation_data=(x_test,y_test),
          callbacks=[cb])  # Pass callback to training

In [ ]:
model = create_model()
model.load_weights(checkpoint_path.format(epoch=10))
# Re-evaluate the model
loss, acc = model.evaluate(x_test,  y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Using SavedModel Objects

[saved_model guide](https://www.tensorflow.org/guide/saved_model)

`tf.saved_model.save(pretrained_model, "/tmp/mobilenet/1/")` Uses the TF Serving convention that the last path directory should be the version number of the model.

Can load the SavedModel back into Python with `tf.saved_model.load(path)`

The `saved_model.pb` file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs.

The `variables` directory contains a standard training checkpoint 

The `assets` directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.

[SavedModels from TF Hub in TensorFlow 2](https://www.tensorflow.org/hub/tf2_saved_model)

[Reusable SavedModels](https://www.tensorflow.org/hub/reusable_saved_models)

## Concrete Functions
[Concrete functions](https://www.tensorflow.org/guide/concrete_function)

## Converting TF2 Models to PyTorch
[From TensorFlow to PyTorch](https://medium.com/huggingface/from-tensorflow-to-pytorch-265f40ef2a28)

[Converting a Simple Deep Learning Model from PyTorch to TensorFlow](https://towardsdatascience.com/converting-a-simple-deep-learning-model-from-pytorch-to-tensorflow-b6b353351f5d)

[Converting Tensorflow Checkpoints](https://huggingface.co/transformers/converting_tensorflow_models.html)